<a href="https://colab.research.google.com/github/matteobpunto/marketing_ads_predictor/blob/main/Marketing_Ads_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CREAZIONE DEL DATABASE**

In [ ]:
import mysql.connector
import csv

# Connect to the MySQL server
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password=""
)

cursor = conn.cursor()

# Create SQL database
cursor.execute("CREATE DATABASE IF NOT EXISTS marketing_adv")

cursor.execute("USE marketing_adv")

cursor.execute("""
CREATE TABLE IF NOT EXISTS company (
    company_id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(50),
    office VARCHAR(50),
    n_employees INT
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS marketing (
    marketing_id INT PRIMARY KEY AUTO_INCREMENT,
    tv FLOAT,
    radio FLOAT,
    newspaper FLOAT,
    sales FLOAT,
    company_id INT,
    FOREIGN KEY (company_id) REFERENCES company(company_id)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS type (
    type_id INT PRIMARY KEY AUTO_INCREMENT,
    spot BOOL,
    inspot BOOL,
    teleshopping BOOL,
    videoclip BOOL,
    trailer BOOL
    );
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS slot (
    slot_id INT PRIMARY KEY AUTO_INCREMENT,
    time_slot VARCHAR(20)
    );
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS broadcaster (
    broadcaster_id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(50),
    network VARCHAR(50)
    );
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS target (
    target_id INT PRIMARY KEY AUTO_INCREMENT,
    age_range VARCHAR(20) NOT NULL
    );
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS marketing_tv (
    tv_id INT PRIMARY KEY,
    marketing_id INT,
    type_id INT,
    slot_id INT,
    broadcaster_id INT,
    target_id INT,
    FOREIGN KEY (type_id) REFERENCES type(type_id),
    FOREIGN KEY (slot_id) REFERENCES slot(slot_id),
    FOREIGN KEY (broadcaster_id) REFERENCES broadcaster(broadcaster_id),
    FOREIGN KEY (target_id) REFERENCES target(target_id),
    FOREIGN KEY (marketing_id) REFERENCES marketing(marketing_id)
);
""")

# ***INSERIMENTO COMPANY***

In [ ]:
import csv
import mysql.connector

# Connect to the MySQL server
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password=""
)

cursor = conn.cursor()

cursor.execute("USE marketing_adv")

with open('Aziende_Italiane.csv', encoding='utf-8') as f:
    lettore = csv.reader(f, delimiter=',')
    f.readline()

    lista_prova = []
    for riga in lettore:

        query = f"INSERT INTO company (name, office, n_employees) VALUES ('{riga[0]}','{riga[1]}', {riga[2]})"
        print(query)

        cursor.execute(query)

        lista_prova.append(query)

conn.commit()

print("The company data has been succesfully inserted!")
print(lista_prova)

cursor.close()
conn.close()

# **INSERIMENTO MARKETING**

In [ ]:
import csv
import mysql.connector
import random

# Connect to the MySQL server
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="marketing_adv"  # Set the database directly in the connection
)

cursor = conn.cursor()

# Get all available company_id values from the company table
cursor.execute("SELECT company_id FROM company")
company_ids = cursor.fetchall()
for row in company_ids:
    company_id = row[0]

# Open the CSV file
with open('Advertising_clear.csv', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    f.readline()

    query = """
    INSERT INTO marketing (tv, radio, newspaper, sales, company_id)
    VALUES (%s, %s, %s, %s, %s)
    """

    data_to_insert = []
    for row in reader:
        tv = float(row[0])
        radio = float(row[1])
        newspaper = float(row[2])
        sales = float(row[3])
        company_id = random.choice(company_ids)[0]

        data_to_insert.append((tv, radio, newspaper, sales, company_id))

    cursor.executemany(query, data_to_insert)
    conn.commit()
print("Marketing data has been successfully inserted!")

# Close the cursor and connection
cursor.close()
conn.close()

# **INSERIMENTO SLOT**

In [ ]:
import mysql.connector

# Connect to the database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="marketing_adv"
)

cursor = conn.cursor()

# Define time slots
time_slots = [
    ('00:00', '04:00', '04:00', '08:00', '08:00', '12:00', '12:00', '16:00', '16:00', '20:00', '20:00', '00:00')
]

# SQL query to insert data
query = """
INSERT INTO slot (slot1_start, slot1_end, slot2_start, slot2_end,
                  slot3_start, slot3_end, slot4_start, slot4_end,
                  slot5_start, slot5_end, slot6_start, slot6_end)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Execute the query
cursor.executemany(query, time_slots)

# Commit the transaction
conn.commit()

print("Time slot data has been successfully inserted!")

# Close the cursor and connection
cursor.close()
conn.close()

# **INSERIMENTO TARGET**

In [ ]:
import mysql.connector

# Connect to the database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="marketing_adv"
)

cursor = conn.cursor()

# SQL query to insert data into the 'target' table
query = "INSERT INTO target (age_range) VALUES (%s)"
values = [
    ('under 18',),
    ('18 - 25',),
    ('25 - 40',),
    ('40 - 60',),
    ('over 60',)
]

cursor.executemany(query, values)

# Commit the changes to the database
conn.commit()

print("Target data has been succesfully inserted!")

# Close the cursor and connection
cursor.close()
conn.close()

# **INSERIMENTO BROADCASTER**

In [ ]:
import csv
import mysql.connector

# Connect to the MySQL server
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password=""
)

cursor = conn.cursor()

cursor.execute("USE marketing_adv")

with open('../../../Downloads/Reti_Televisive_Italiane_Uniche.csv', encoding='utf-8') as f:
    lettore = csv.reader(f, delimiter=',')
    f.readline()

    lista_prova = []
    for riga in lettore:

        query = f"INSERT INTO broadcaster (name, network) VALUES ('{riga[0]}','{riga[1]}')"
        print(query)

        cursor.execute(query)

        lista_prova.append(query)

conn.commit()

print("Broadcoaster`s data has been successfully inserted!")
print(lista_prova)

cursor.close()
conn.close()

# **INSERIMENTO TYPE**

In [ ]:
import csv
import mysql.connector
import random
from mysql.connector import Error

def recupera_dati_lista_int(query):
    try:
        # Conection to the database
        connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="",
            database="marketing_adv"
        )
        if connection.is_connected():
            cursor = connection.cursor()  # Returns results as dictionaries

            # Execution of the query
            cursor.execute(query)

            # Dataa retrival
            result = [elem[0] for elem in cursor.fetchall()]

            cursor.close()
            return result
    except Error as e:
        print(f"Error while executing the query: {e}")
        return None
    finally:
        if connection.is_connected():
            connection.close()

# Connection to the MySQL server
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password=""
)

cursor = conn.cursor()

cursor.execute("USE marketing_adv")

lista_bool = [True, False]
lista_tipi = ["spot", "inspot", "teleshopping", "videoclip", "trailer"]

tipo_scelto = random.choice(lista_tipi)
indice = lista_tipi.index(tipo_scelto)

colonne = recupera_dati_lista_int("SELECT COUNT(*) FROM marketing")

for _ in range(189):
    stringa = ''

    for x in range(len(lista_tipi)):
        if x == indice:
            stringa += 'TRUE'
        else:
            stringa += 'FALSE'

        if x != len(lista_tipi) - 1:
            stringa += ', '

    query = (f"INSERT INTO type(`spot`, `inspot`, `teleshopping`, `videoclip`, `trailer`)"
             f" VALUES ({stringa})")

    cursor.execute(query)
    conn.commit()

    print("Type data has been succesfully inserted!")

    cursor.close()
    conn.close()

# **INSERIMENTO MARKETING TV**

In [ ]:
# Importing the libraries we'll use
import mysql.connector
import random
from marketing_ads_predictor.database2.insert_type import recupera_dati_lista_int

# Connection to the mySQL server
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password=""
)

# Creation of a cursor to navigate our database
cursor = conn.cursor()

# Selecting the right database
cursor.execute("USE marketing_adv")

# Using the mySQL language for the insert of marketing values

insertInto = ("INSERT INTO `marketing_tv`(`marketing_id`, `type_id`, `slot_id`, `broadcaster_id`, `target_id`)"
              " VALUES ('[value-2]','[value-3]','[value-4]','[value-5]','[value-6]')")

# Taking all the marketing id`s from marketing and order them by their id
cursor.execute("SELECT marketing_id FROM marketing ORDER BY `marketing`.`marketing_id` ASC")
Marketing_list_id = cursor.fetchall()
print(type(Marketing_list_id))
print(Marketing_list_id)

# Creating dictionaries for age groups and percentages of their weight
age_groups = {
    1: "under 18",
    2: "18 - 25",
    3: "25 - 40",
    4: "40 - 60",
    5: "over 60"
}

percentages = {
    1: 30,  # 10% under 18
    2: 5,  # 20% 18-25
    3: 15,  # 25% 25-40
    4: 20,  # 15% 40-60
    5: 30  # 30% over 60
}

# Counting all data from marketing and converting them into percentages
num_samples = 189
weights = [percentages[key] / 100 for key in age_groups]

# Creating a random list using random function to determine the percentages
random_list = random.choices(list(age_groups.keys()), weights=weights, k=num_samples)

print(random_list)
print(len(random_list))

# Creating dictionaries for time slot and a dictionary
time_slots = {
    "18 - 25": ["00:00 - 04:00"],
    "25 - 40": ["04:00 - 08:00"],
    "over 60": ["08:00 - 12:00"],
    "under 18": ["12:00 - 16:00", "16:00 - 20:00"],
    "40 - 60": ["20:00 - 00:00"]
}

Slot_dictionary = {
    "00:00 - 04:00": 1,
    "04:00 - 08:00": 2,
    "08:00 - 12:00": 3,
    "12:00 - 16:00": 4,
    "16:00 - 20:00": 5,
    "20:00 - 00:00": 6
}

valuesInsertInto = ""
row_count = 0
for i, num in enumerate(random_list):
    # id taken from marketing table
    marketingID = i + 1
    # id taken from type table
    typeID = i + 1

    # id target table taken from list made with the weighted percentages
    targetID = num
    print(marketingID)
    print(num)
    targetinstringa = age_groups.get(targetID)
    slot = time_slots.get(targetinstringa)
    slotgiusto = random.choice(slot)
    slotID = Slot_dictionary.get(slotgiusto)
    broadcasterID = random.randint(1, 10)

    valuesInsertInto = f"{marketingID}, {typeID}, {slotID}, {broadcasterID}, {targetID}"

    query = f"""INSERT INTO `marketing_tv`(`marketing_id`, `type_id`, `slot_id`, `broadcaster_id`, `target_id`)
               VALUES ({valuesInsertInto})"""

    cursor.execute(query)
    row_count += 1

# Commit of the modifications
conn.commit()

print(f"{row_count} rows have been succesfully inserted!")

# Connection closed
cursor.close()
conn.close()